・担当者名ごとのエクセルファイルを作成→アレンジ：集計ファイルにピボットテーブルシートを追加  
・各ファイルには担当者ごとにどの取引先に対して何をいくら売ったのか、全ての月の合計値でピボットテーブルに落とし込む

In [57]:
import pandas as pd
import openpyxl as px

#4-6月のシートをconcat→df_allMonのdfに格納
WB=pd.ExcelFile("月別売上.xlsx")
ws_list=WB.sheet_names
for i in range(len(ws_list)):
    if "月" in ws_list[i]:
        continue 
    else:
        ws_list.pop(i) #"*月"シートのリスト「ws_list」を取得

df_allMon=pd.DataFrame()
for ws in ws_list:
    df=pd.read_excel("月別売上.xlsx",sheet_name=ws)
    df_allMon=pd.concat([df_allMon,df])

#社員番号をキーに担当者名列と合計金額列を追加
wb=px.load_workbook("社員リスト.xlsx")
ws=wb["リスト"]
master_lastr=0
for r in reversed(range(ws.max_row+1)):
    if ws.cell(r,1).value!=None:
        master_lastr=r
        break  #マスタシートの最終行を取得（r）

for r2 in range(2,master_lastr+1):
    tgt=ws.cell(r2,1).value
    if "-" in tgt:
        index=tgt.find("-")
        ws.cell(r2,1).value=int(tgt.replace(tgt[:index+1],""))
wb.save("社員リスト（改）.xlsx") #マスタの社員番号を修正して「社員リスト（改）」ファイルに格納

df_master=pd.read_excel("社員リスト（改）.xlsx",sheet_name=0)
df_re=pd.merge(df_allMon,df_master,on="社員番号")
df_re=df_re.rename(columns={"名前":"担当者"})
df_re=df_re.iloc[:,[0,1,2,7,4,5,6]]
df_re["売上金額"]=df_re["単価"]*df_re["個数"]

#担当者ごとのピボットテーブルを作成しシートとして追加
df_tantou=pd.DataFrame()
name_list=df_re["担当者"].unique()
for nm in name_list:
    df_tantou=df_re[df_re["担当者"]==nm]
    df_tantou=df_tantou.pivot_table(index="取引先",columns="商品名",values="売上金額",aggfunc=sum,fill_value=0)
    df_tantou.loc["合計"]=df_tantou.sum(numeric_only=True)
    df_tantou["合計"]=df_tantou["PC"]+df_tantou["キーボード"]+df_tantou["タブレット"]+df_tantou["マウス"]
    with pd.ExcelWriter("月別売上 - コピー.xlsx",mode="a") as writer:
        df_tantou.to_excel(writer,sheet_name=nm)
    

In [66]:
df_syain=pd.read_excel("社員リスト.xlsx",sheet_name=0)
df_syain["社員番号"]=df_syain["社員番号"].str.replace("[A-Z]-","",regex=True)
df_syain

,社員番号,名前
0,1100000,田中
1,1200000,佐藤
2,1300000,鈴木
3,1400000,中村


In [76]:
import pandas as pd
import openpyxl as px

#4-6月のシートをconcat→df_allMonのdfに格納
WB=pd.ExcelFile("月別売上.xlsx")
ws_list=WB.sheet_names
for i in range(len(ws_list)):
    if "月" in ws_list[i]:
        continue 
    else:
        ws_list.pop(i) #"*月"シートのリスト「ws_list」を取得

df_allMon=pd.DataFrame()
for ws in ws_list:
    df=pd.read_excel("月別売上.xlsx",sheet_name=ws)
    df_allMon=pd.concat([df_allMon,df])

#社員番号をキーに担当者名列と合計金額列を追加
wb=px.load_workbook("社員リスト.xlsx")
ws=wb["リスト"]
master_lastr=0
for r in reversed(range(ws.max_row+1)):
    if ws.cell(r,1).value!=None:
        master_lastr=r
        break  #マスタシートの最終行を取得（r）

df_master=pd.read_excel("社員リスト.xlsx",sheet_name=0)
df_master["社員番号"]=df_master["社員番号"].str.replace("[A-Z]-","",regex=True)
df_master=df_master.astype({"社員番号":int})

df_re=pd.merge(df_allMon,df_master,on="社員番号")
df_re=df_re.rename(columns={"名前":"担当者"})
df_re=df_re.iloc[:,[0,1,2,7,4,5,6]]
df_re["売上金額"]=df_re["単価"]*df_re["個数"]

#担当者ごとのピボットテーブルを作成しシートとして追加
df_tantou=pd.DataFrame()
name_list=df_re["担当者"].unique()
for nm in name_list:
    df_tantou=df_re[df_re["担当者"]==nm]
    df_tantou=df_tantou.pivot_table(index="取引先",columns="商品名",values="売上金額",aggfunc="sum",fill_value=0,margins=True,margins_name="総計")
    with pd.ExcelWriter("月別売上 - コピー.xlsx",mode="a") as writer:
        df_tantou.to_excel(writer,sheet_name=nm)
    